### Import libraries

In [ ]:
#import libraries
from aitk.utils import gallery, array_to_image
from aitk.networks import Network
from time import sleep

import tensorflow
import time
from tensorflow.keras.datasets import fashion_mnist #using fasion mnist dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'aitk'

### Get data

In [ ]:
#10 categories of mnist fasion dataset: 0=t-shirt, 1=trouser, 2=pullover, 3=dress, 4=coat, 5=sandal, 6=shirt, 7=sneaker, 8=bag, 9=ankle boot
(train_x, train_y),(test_x, test_y)=fashion_mnist.load_data()
train_x.shape #60000 images of 28x28 arrays
test_x.shape #10000 images of 28x28 arrays
train_y[:20] #returns first 20 outputs of training data (0-9, one digit for each category)

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9, 1, 0, 6, 4],
      dtype=uint8)

### Prepare data

In [ ]:
#get min and max input
min_input=train_x.min()
max_input=train_x.max()
print("range of input values is:", min_input, max_input)

#normalize input data so that they are in range [0,1]
train_x_normalized=(train_x-min_input)/float(max_input-min_input)
test_x_normalized=(test_x-min_input)/float(max_input-min_input)

#reshape input data to contain channels (1 for Grayscale, 3 for RBG)
#this is the final input data we will use for our networks
train_x_normalized=train_x_normalized.reshape(60000,28,28,1)
test_x_normalized=test_x_normalized.reshape(10000,28,28,1)

#convert ouputs to one-hot vectors (ex. 9 becomes [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.])
num_categories=len(set(train_y)) #gets number of unique outputs
train_y_category=to_categorical(train_y,num_categories) #convert training ouputs to one-hot vectors
test_y_category=to_categorical(test_y,num_categories) #convert testing ouputs to one-hot vectors
train_y_category[0]

range of input values is: 0 255


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

### Construct and compile multi-layer dense neural network

In [ ]:
#construct multi-layer dense neural network with RELU activation function
ml_nn=Sequential()
ml_nn.add(Flatten(input_shape=(28,28,1)))
ml_nn.add(Dense(100,activation='relu'))
ml_nn.add(Dense(10,activation='softmax'))
ml_nn.summary()

#compile network with stoachastic gradient descent and loss function
ml_nn.compile(optimizer="SGD",loss="categorical_crossentropy",metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


2022-04-01 03:58:53.153792: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NOT_INITIALIZED: initialization error
2022-04-01 03:58:53.153839: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gpu01.swarthmore.edu
2022-04-01 03:58:53.153844: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gpu01.swarthmore.edu
2022-04-01 03:58:53.153967: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-04-01 03:58:53.153992: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-04-01 03:58:53.153996: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-04-01 03:58:53.154372: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU 

### Construct convolutional neural network

In [ ]:
#construct convolutional neural network with RELU activation function
c_nn=Sequential()
c_nn.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
c_nn.add(MaxPooling2D(pool_size=(2,2)))
c_nn.add(Flatten())
c_nn.add(Dense(128,activation='relu'))
c_nn.add(Dense(10,activation='softmax'))
c_nn.summary()

#compile network with stoachastic gradient descent and loss function
c_nn.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
________________________________________________

### Create an aitk network for dense nn for visualization tools

In [ ]:
ml_net=Network(ml_nn) #reconstruct neural network
for i in range(10):
    ml_net.propagate(train_x_normalized[i])
    ml_net.display(train_x_normalized[i])
#     sleep(1.0)

### Create an aitk network for convolutional nn for visualization tools

In [ ]:
c_net=Network(c_nn) #reconstruct neural network
for i in range(10):
    c_net.propagate(train_x_normalized[i])
    c_net.display(train_x_normalized[i])
#     sleep(1.0)

### Train the dense nn model

In [ ]:

# history_ml=ml_net.fit(train_x_normalized,train_y_category,verbose=1,validation_data=(test_x_normalized,test_y_category),epochs=5)

start = time.time()
history_ml=ml_net.fit(train_x_normalized,train_y_category,verbose=2,epochs=5)
print("training time: ",time.time() - start)


start = time.time()
history_ml=ml_net.evaluate(test_x_normalized,test_y_category)
print("Evaluation time: ",time.time() - start)


Epoch 5/5 loss: 0.4266102910041809 - accuracy: 0.8521000146865845
training time:  9.798975229263306
313/313 [==============================] - 0s 792us/step - loss: 0.4515 - accuracy: 0.8413
Evaluation time:  0.450939416885376


### Train the convolutional nn model

In [ ]:
# history_c=c_net.fit(train_x_normalized,train_y_category,verbose=1,validation_data=(test_x_normalized,test_y_category),epochs=5)


start = time.time()
history_ml=c_net.fit(train_x_normalized,train_y_category,verbose=2,epochs=5)
print("training time: ",time.time() - start)


start = time.time()
history_ml=c_net.evaluate(test_x_normalized,test_y_category)
print("Evaluation time: ",time.time() - start)

Epoch 5/5 loss: 0.1631983518600464 - accuracy: 0.9401333332061768
training time:  43.248151540756226
313/313 [==============================] - 1s 2ms/step - loss: 0.2698 - accuracy: 0.9070
Evaluation time:  0.6639385223388672


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=803972cf-e988-4b15-bfb5-5d257db67f52' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>